In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import panel as pn
pn.extension('plotly')
import plotly.express as px

In [2]:
#Find the data directory
datadir = '/raid/nmis-ddm/2021-10-11--radial-forge-mini-missy'
filelist = os.listdir(datadir)

In [3]:
# Sepreate cold and hot forging csv files
hotlist =[]
coldlist =[]
for file in filelist:
    if file.split("_")[6] == 'COLD':
        coldlist.append(file)
    else:
        hotlist.append(file) 
        
# function for sorting files in both cold list and hot list
def sortKeyFunc(x):
    return (x[-6:])

# Sorting both cold and hot forging files
coldlist.sort(key = sortKeyFunc)
hotlist.sort(key = sortKeyFunc)

print(coldlist)

['AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_01.csv', 'AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_02.csv', 'AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_03.csv', 'AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_04.csv', 'AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_05.csv', 'AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_06.csv', 'AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_07.csv', 'AFRC_DIRF_2022_Mini_MISSY_20211011_COLD_08.csv']


In [4]:
# User defined filename chosen from coldlist or hotlist
filename =coldlist[0]
df = pd.read_csv(datadir+'/'+filename, encoding = "ISO-8859-1")
df.head()

,Timer Tick [ms],Program-Nr,Power [kW],Force [kN],A_ges_vibr,Stroke [1/min],EXZ_pos [deg],hydraulic low pressure [bar],hydraulic high pressure [bar],A_ACTpos [mm],...,c02w,Frc_Volt.1,U_ACTVAL_2,U_NOMVAL,$E_GH_FAULT_1 (I25S21),$E_GH_FAULT_2 (I26S21),$U_GH_HEATON_1 (U25S0).1,$U_GH_NOMEXT_1 (U25S1),$U_GH_HEATON_2 (U26S0),$U_GH_NOMEXT_2 (U26S1)
0,340733338,290,-0.117223,0.0,2.659624e-08,0,179.212578,-0.040595,192.303834,1219.0,...,0,1.015625,0,500,0,0,0,1,0,1
1,340733348,290,-0.117015,0.0,6.128116e-09,0,179.212578,-0.040152,192.275848,1219.0,...,0,1.015625,0,500,0,0,0,1,0,1
2,340733358,290,-0.116253,0.0,7.120765e-09,0,179.212578,-0.039790,192.281570,1219.0,...,0,1.015625,0,500,0,0,0,1,0,1
3,340733368,290,-0.116064,0.0,7.961022e-09,0,179.212578,-0.039494,192.340023,1219.0,...,0,1.010742,0,500,0,0,0,1,0,1
4,340733378,290,-0.116854,0.0,8.672281e-09,0,179.212578,-0.038384,192.360095,1219.0,...,0,1.015625,0,500,0,0,0,1,0,1


In [5]:
# Create a radio button widget for the panel
radio_buttons = pn.widgets.RadioButtonGroup(options=['Cold', 'Hot'])

# Create a slider widget for the panel
slider = pn.widgets.IntSlider(start=0, end=600, value=10, name='time range (seconds)')

def plot_data(df, miliseconds,y):
    # Plotting in miliseconds
    if miliseconds:
        # Starting from zero time stamp
        df["Timer Tick from Zero [ms]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])
        fig = px.line(df, x="Timer Tick from Zero [ms]", y="Power [kW]", title='Forge data - power')


    else:
        # Starting from zero time stamp
        df["Timer Tick from Zero [s]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])/1000
        time_start = df['Timer Tick from Zero [s]'].iloc[0]
        time_end = df['Timer Tick from Zero [s]'].iloc[y*100]
        df_time_period = df.loc[(df['Timer Tick from Zero [s]']>time_start) & (df['Timer Tick from Zero [s]']<time_end)]
        fig = px.line(df_time_period, x="Timer Tick from Zero [s]", y="Power [kW]", title='Forge data - power')    
    
    return fig

@pn.depends(radio_buttons)
def plot_forge_data(x,y, miliseconds = False):

    if x == 'Hot':
        # Selecting filenames with hot forging data
        
        filename = hotlist[0]
        df = pd.read_csv(datadir+'/'+filename, encoding = "ISO-8859-1")
        
        plot_b = plot_data(df,miliseconds,y)

        return plot_b
    if x == 'Cold':
        
        # Selecting filenames with cold forging data
        filename = coldlist[0]
        df = pd.read_csv(datadir+'/'+filename, encoding = "ISO-8859-1")
        plot_b = plot_data(df, miliseconds,y)
        
        return plot_b

In [6]:
# Plot total number of employess and turnover for each sector(depending on SIC code) 
forge_db_explanation_pane = pn.pane.Markdown("""
# cold and hot forge data
""", width=200)

# Create interactive panels with radio buttons and checkboxes
forge_db_chart_interact = pn.interact(plot_forge_data, x=radio_buttons, y=slider)
forge_db_layout = pn.Row(forge_db_explanation_pane, forge_db_chart_interact)
forge_db_layout.servable()

Row
    [0] Markdown(str, width=200)
    [1] Column
        [0] Column
            [0] RadioButtonGroup(options=['Cold', 'Hot'], value='Cold')
            [1] IntSlider(end=600, name='time range (seconds)', value=10)
            [2] Checkbox(name='miliseconds')
        [1] Row
            [0] Plotly(Figure, name='interactive00105')

In [7]:
   
df_stacked = []
for file in coldlist:
    df = pd.read_csv(datadir+'/'+file, encoding = "ISO-8859-1")
    df['filename']= file
    if len(df_stacked) == 0:
        df_stacked = df
    else:    
        df_stacked = df_stacked.append(df)

# List of unique filenames
filename = list(df_stacked.filename.unique())

# create widget for drop down menu
select = pn.widgets.Select(name = 'Select file', options = filename)


# Create a slider widget for the panel
slider = pn.widgets.IntSlider(start=0, end=600, value=10, name='time range (seconds)')

def plot_data(df, miliseconds,y):
    # Plotting in miliseconds
    if miliseconds:
        # Starting from zero time stamp
        df["Timer Tick from Zero [ms]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])
        fig = px.line(df, x="Timer Tick from Zero [ms]", y="Power [kW]", title='Forge data - power')


    else:
        # Starting from zero time stamp
        df["Timer Tick from Zero [s]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])/1000
        time_start = df['Timer Tick from Zero [s]'].iloc[0]
        time_end = df['Timer Tick from Zero [s]'].iloc[y*100]
        df_time_period = df.loc[(df['Timer Tick from Zero [s]']>time_start) & (df['Timer Tick from Zero [s]']<time_end)]
        fig = px.line(df_time_period, x="Timer Tick from Zero [s]", y="Power [kW]", title='Forge data - power')    
    
    return fig

def plot_forge_data(y,filename,miliseconds = False):
        
    # Read files with cold forging data
    df = pd.read_csv(datadir+'/'+filename, encoding = "ISO-8859-1")
    plot_b = plot_data(df, miliseconds,y)
        
    return plot_b

/tmp/ipykernel_2817401/2117063574.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/2117063574.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/2117063574.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/2117063574.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/2117063574.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/2117063574.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas

In [8]:
# Plot total number of employess and turnover for each sector(depending on SIC code) 
forge_db_explanation_pane = pn.pane.Markdown("""
# cold and hot forge data
""", width=200)

# Create interactive panels with radio buttons and checkboxes
forge_db_chart_interact = pn.interact(plot_forge_data, y=slider, filename = select)
forge_db_layout = pn.Row(forge_db_explanation_pane, forge_db_chart_interact)
forge_db_layout.servable()

Row
    [0] Markdown(str, width=200)
    [1] Column
        [0] Column
            [0] IntSlider(end=600, name='time range (seconds)', value=10)
            [1] Select(options=['AFRC_DIRF_2022_Mini_MISS...], value='AFRC_DIRF_2022_Mini_MISSY...)
            [2] Checkbox(name='miliseconds')
        [1] Row
            [0] Plotly(Figure, name='interactive00123')

In [9]:
def stack_files(filelist):    
    df_stacked = []
    for file in filelist:
        df = pd.read_csv(datadir+'/'+file, encoding = "ISO-8859-1")
        df['filename']= file
        if len(df_stacked) == 0:
            df_stacked = df
        else:    
            df_stacked = df_stacked.append(df)
    return df_stacked
# Create a radio button widget for the panel
radio_buttons = pn.widgets.RadioButtonGroup(options=['Cold', 'Hot'])

# Create a slider widget for the panel
slider = pn.widgets.IntSlider(start=0, end=600, value=10, name='time range (seconds)')

def plot_data(filelist, miliseconds,y):

    # Plotting in miliseconds
    if miliseconds:
        
        df_stacked = []
        for file in filelist:
            df = pd.read_csv(datadir+'/'+file, encoding = "ISO-8859-1")
            df['filename']= file
            # Starting from zero time stamp
            df["Timer Tick from Zero [ms]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])
            if len(df_stacked) == 0:
                df_stacked = df
            else:    
                df_stacked = df_stacked.append(df)
        
        fig = px.line(df_stacked, x="Timer Tick from Zero [ms]", y="Power [kW]", color = "filename", title='Forge data - power',height=500, width=1200)


    else: 
        df_stacked = []
        for file in filelist:
            df = pd.read_csv(datadir+'/'+file, encoding = "ISO-8859-1")
            df['filename']= file
            # Starting from zero time stamp
            df["Timer Tick from Zero [s]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])/1000
            time_start = df['Timer Tick from Zero [s]'].iloc[0]
            time_end = df['Timer Tick from Zero [s]'].iloc[y*100]
            df_time_period = df.loc[(df['Timer Tick from Zero [s]']>time_start) & (df['Timer Tick from Zero [s]']<time_end)]
            if len(df_stacked) == 0:
                df_stacked = df_time_period
            else:    
                df_stacked = df_stacked.append(df_time_period)
        
        fig = px.line(df_stacked, x="Timer Tick from Zero [s]", y="Power [kW]", color = "filename", title='Forge data - power',height=500, width=1200)    
    
    return fig

@pn.depends(radio_buttons)
def plot_forge_data(x,y, miliseconds = False):

    if x == 'Hot':
        # Files with hot forging data
        plot_b = plot_data(hotlist,miliseconds,y)

        return plot_b
    if x == 'Cold':
        
        # Files with cold forging data
        plot_b = plot_data(coldlist,miliseconds,y)

        return plot_b
         

In [10]:
# Plot total number of employess and turnover for each sector(depending on SIC code) 
forge_db_explanation_pane = pn.pane.Markdown("""
# cold and hot forge data
""", width=200)

# Create interactive panels with radio buttons and checkboxes
forge_db_chart_interact = pn.interact(plot_forge_data, x=radio_buttons, y=slider)
forge_db_layout = pn.Row(forge_db_explanation_pane, forge_db_chart_interact)
forge_db_layout.servable()

/tmp/ipykernel_2817401/3698131468.py:49: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3698131468.py:49: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3698131468.py:49: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3698131468.py:49: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3698131468.py:49: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3698131468.py:49: FutureWarning:

The frame.append method is deprecated and will be removed from 

Row
    [0] Markdown(str, width=200)
    [1] Column
        [0] Column
            [0] RadioButtonGroup(options=['Cold', 'Hot'], value='Cold')
            [1] IntSlider(end=600, name='time range (seconds)', value=10)
            [2] Checkbox(name='miliseconds')
        [1] Row
            [0] Plotly(Figure, name='interactive00140')

In [19]:
def df_filter(df):
    df_filtered = df.drop(columns=['Program-Nr', 'A_ges_vibr','B_ACTpos [mm]','D_ACTpos [mm]','B_ACT_Force [kN]','D_ACT_Force [kN]','B_ACTspd [mm/min]','D_ACTspd [mm/min]',
                               'INDA_NOMpos [deg]','B_NOMpos [mm]','D_NOMpos [mm]', 'INDA_OUT [%]','B_OUT [%]','D_OUT [%]','INDA_ACTspd [U/min]','B_NOMspd [mm/min]',
                               'D_NOMspd [mm/min]','A_NOM_Force [kN]', 'B_NOM_Force [kN]','DB_NOM_Force [kN]','D_NOM_Force [kN]','$F_F41L (I14S8)','Feedback B [%]',
                               'Feedback D [%]','RamRetract_ActSpd [rpm]','$U_GH_HEATON_1 (U25S0)','W2 Waterflow [l]','W1 Waterflow [l]','$E_GH_FAULT_1 (I25S21)', 
                               '$B12R_Y11 (U14S16)','O_EMERG','STP || EM', 'O_MASTOP','$E_GH_FAULT_2 (I26S21)','c02w','$U_GH_HEATON_1 (U25S0).1', 
                               '$U_GH_NOMVAL_1 (U25W1)','$U_GH_NOMEXT_1 (U25S1)', '$U_GH_HEATON_2 (U26S0)', '$U_GH_NOMEXT_2 (U26S1)'])
    df_filtered["Timer Tick from Zero [s]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])/1000
    df_filtered["Timer Tick from Zero [ms]"] = (df["Timer Tick [ms]"]-df['Timer Tick [ms]'].iloc[0])
    return (df_filtered)
    
# Create a radio button widget for the panel
radio_buttons = pn.widgets.RadioButtonGroup(options=['Cold', 'Hot'])

# Create a slider widget for the panel
slider = pn.widgets.IntSlider(start=0, end=600, value=10, name='time range (seconds)')

# Create widget for drop down menu
file = coldlist[0]
df = pd.read_csv(datadir+'/'+file, encoding = "ISO-8859-1")
df_filtered = df_filter(df)
cols = list(df_filtered.columns)

x = pn.widgets.Select(name='X-axis', options=cols, value='Timer Tick from Zero [s]')
y = pn.widgets.Select(name='Y-axis', options=cols ,value='Power [kW]')


def plot_data(filelist,slider_pos, x, y):

    # Plotting in miliseconds 
    df_stacked = []
    for file in filelist:
        df = pd.read_csv(datadir+'/'+file, encoding = "ISO-8859-1")
        df['filename']= file
        df_filtered = df_filter(df)
        time_start = df_filtered['Timer Tick from Zero [s]'].iloc[0]
        time_end = df_filtered['Timer Tick from Zero [s]'].iloc[slider_pos*100]
        df_time_period = df_filtered.loc[(df_filtered['Timer Tick from Zero [s]']>time_start) & (df_filtered['Timer Tick from Zero [s]']<time_end)]
        if len(df_stacked) == 0:
            df_stacked = df_time_period
        else:    
            df_stacked = df_stacked.append(df_time_period)

    fig = px.scatter(df_stacked,x =x, y=y, color = "filename", height=500, width=1500)    
    
    return fig

@pn.depends(radio_buttons,x.param.value, y.param.value)
def plot_forge_data(category,x,y,slider_pos):

    if category == 'Hot':
        # Files with hot forging data
        plot_b = plot_data(hotlist,slider_pos,x,y)

        return plot_b
    if category == 'Cold':
        
        # Files with cold forging data
        plot_b = plot_data(coldlist,slider_pos,x,y)

        return plot_b
         

In [20]:
# Create interactive panels with radio buttons and checkboxes
explanation_pane = pn.pane.Markdown("""
# Data Exploration
""", width=800)
forge_db_chart_interact = pn.interact(plot_forge_data,x= x,y =y, category=radio_buttons, slider_pos=slider)
forge_db_layout = pn.Column(explanation_pane,forge_db_chart_interact)
forge_db_layout.servable()

/tmp/ipykernel_2817401/3885373479.py:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3885373479.py:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3885373479.py:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3885373479.py:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3885373479.py:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_2817401/3885373479.py:42: FutureWarning:

The frame.append method is deprecated and will be removed from 

Column
    [0] Markdown(str, width=800)
    [1] Column
        [0] Column
            [0] RadioButtonGroup(options=['Cold', 'Hot'], value='Cold')
            [1] Select(name='X-axis', options=['Timer Tick [ms]', ...], value='Timer Tick from Z...)
            [2] Select(name='Y-axis', options=['Timer Tick [ms]', ...], value='Power [kW]')
            [3] IntSlider(end=600, name='time range (seconds)', value=10)
        [1] Row
            [0] Plotly(Figure, name='interactive00236')